# Paso 2.
Obteniendo espigas

In [1]:
function ESU( ch::Array, thr::Real, d::Int )
    #= Se obtienen los frames y voltajes que sobrepasan el umbral establecido. 
    Si hay eventos supraumbral a d-frames de distancia, se selecciona aquel que 
    tenga menor voltaje. =#
    OK = findall( ch .<= thr );
    if !isempty(OK)
        init = 1;
        while init == 1
            A = OK[ 1:( end - 1 ) ]; B = OK[ 2:end ];
            C = B .- A;
            D = findall( C .<= d ) .+ 1;
            if isempty( D )
                init = 0;
            else
                nook = zeros( Int, size( D, 1 ) );
                for E = 1:size( D, 1 )
                    # si el primero es menor que el segundo, quita el segundo
                    if isless( (ch[ OK[ D[ E ] ] ]), (ch[ OK[ D[ E ] - 1 ] ] ) ) 
                        nook[ E ] = D[ E ] - 1;
                    else
                        nook[ E ] = D[ E ];
                    end
                end
                if size( nook, 1 ) > 1
                    OK[ unique( nook ) ] .= 0;
                    filter!( x -> x != 0, OK );
                else
                    OK = OK[ Bool.( OK .!= OK[ nook[ 1 ] ] ) ];
                end
            end
        end
        channelC = zeros( size( OK, 1 ), 1 );
        channelC[ :, 1 ] = OK;
    else
        channelC = [ ];
    end
    return channelC
end
function get_spikes_single( data, sr, stdmin, fcut, frames )
    frames = Int( ceil( frames*sr/1000 ) );
    responsetype = Highpass( fcut, fs=sr ); designmethod = Elliptic( 2, 0.1, 40 );
    xf_eliptic = filt(digitalfilter( responsetype, designmethod ), data );
    thr = -1*stdmin*( median( abs.( xf_eliptic ) )/0.6745 );
    index = Int.(ESU( xf_eliptic, thr, frames ));
    return xf_eliptic, index
end
function cut_spikes( channel, index, w_pre, w_post )
    index = index[ Bool.( 1 .- ( ( index .+ w_post ) .> length( channel ) ) ) ];
    index = index[ Bool.( 1 .- ( ( index .- w_pre ) .< 1 ) ) ];
    spikes = zeros( length( index ), ( w_pre + w_post + 1) );
    lim1 = index .- w_pre; lim2 = index .+ w_post;
    for i = 1:length(index)
        spikes[ i, : ] = channel[ lim1[ i ]:lim2[ i ] ];
    end
    return spikes
end
function espigas_x_ventana(channel, sr, stdmin, fcut, frames, spike_lenght, displace)
    index = [ ];
    responsetype = Highpass( fcut, fs=sr ); 
    designmethod = Elliptic( 2, 0.1, 40 );
    xf_eliptic = filt( digitalfilter( responsetype, designmethod ), channel );
    frames =  Int(floor( ( ( frames*sr )/1000 ) ) );
    spike_lenght = round( ( spike_lenght/1000 )*sr );
    displace = Int( floor( ( displace*sr)/1000 ) );
    i = 1
    init = Int( ( ( i - 1 )*displace + 1 ) );
    endit = Int( init + spike_lenght - 1 );
    while endit <= (length( channel ) - Int(spike_lenght - 1) )
        init = Int( ( ( i - 1 )*displace + 1 ) );
        endit = Int( init + spike_lenght - 1 );
        bin = xf_eliptic[ init:endit ];
        thr = -1*stdmin*( median( abs.( bin ) )/0.6745 );
        index_parcial = Int.( ESU( bin, thr, frames ) );
        if !isempty( index_parcial )
            index_real = index_parcial .+ init .- 1;
            index = vcat( index, index_real );
        end
            i = i + 1
    end
    index = unique( index );
    return xf_eliptic, index
end

espigas_x_ventana (generic function with 1 method)

In [2]:
using JLD, DSP, Statistics

In [3]:
file = "/home/isabel/Desktop/datosprueba/140116s2cut2_evento_2_nosats.jld";
resultsfile = "/home/isabel/Dropbox/Proyects/Metodos_MEA/Scripts/Nuevos/resultados_140116s2cut2_evento_2.jld";

In [4]:
rawdata = load( file )[ "data" ];
vars = load( resultsfile ); 
channels4 = vars[ "coords4" ];

In [5]:
sr = 7022; # sampling rate
window = 5; # ms
displace = 1; # ms
frames = 1; # ms
stdmin = 3; # umbral de deteccion 
fcut = 350; # frecuancias de corte, minima
w_pre = 35; # number of pre-event data points stored
w_post = 35; # number of pos-event data points stored

In [7]:
AllSpikes = Dict( [ ] );
for i = 1:length( channels4 )
    channel = rawdata[ channels4[ i ], : ];
    x, index = espigas_x_ventana( channel, sr, stdmin, fcut, frames, window, displace );
    spikes = cut_spikes( x, index, w_pre, w_post );
    AllSpikes[ string( channels4[ i ] ) ] = [ index, spikes ];
    println(
    "canal: ", channels4[ i ],"...",length( index )," espigas detectadas en canal con ventana deslizante." );
end

canal: 2910...16 espigas detectadas en canal con ventana deslizante.
canal: 2846...20 espigas detectadas en canal con ventana deslizante.
canal: 2847...14 espigas detectadas en canal con ventana deslizante.
canal: 2783...10 espigas detectadas en canal con ventana deslizante.
canal: 2784...16 espigas detectadas en canal con ventana deslizante.
canal: 2848...12 espigas detectadas en canal con ventana deslizante.
canal: 2785...14 espigas detectadas en canal con ventana deslizante.
canal: 2720...13 espigas detectadas en canal con ventana deslizante.
canal: 2721...19 espigas detectadas en canal con ventana deslizante.
canal: 2722...17 espigas detectadas en canal con ventana deslizante.
canal: 2786...7 espigas detectadas en canal con ventana deslizante.
canal: 2723...19 espigas detectadas en canal con ventana deslizante.
canal: 2658...13 espigas detectadas en canal con ventana deslizante.
canal: 2659...13 espigas detectadas en canal con ventana deslizante.
canal: 2660...12 espigas detectadas

canal: 816...12 espigas detectadas en canal con ventana deslizante.
canal: 944...12 espigas detectadas en canal con ventana deslizante.
canal: 943...12 espigas detectadas en canal con ventana deslizante.
canal: 1007...15 espigas detectadas en canal con ventana deslizante.
canal: 1008...12 espigas detectadas en canal con ventana deslizante.
canal: 1071...14 espigas detectadas en canal con ventana deslizante.
canal: 1006...13 espigas detectadas en canal con ventana deslizante.
canal: 1070...11 espigas detectadas en canal con ventana deslizante.
canal: 1134...14 espigas detectadas en canal con ventana deslizante.
canal: 1133...18 espigas detectadas en canal con ventana deslizante.
canal: 1197...15 espigas detectadas en canal con ventana deslizante.
canal: 1198...10 espigas detectadas en canal con ventana deslizante.
canal: 1261...16 espigas detectadas en canal con ventana deslizante.
canal: 1262...15 espigas detectadas en canal con ventana deslizante.
canal: 1325...16 espigas detectadas e

canal: 2219...7 espigas detectadas en canal con ventana deslizante.
canal: 2154...7 espigas detectadas en canal con ventana deslizante.
canal: 2155...11 espigas detectadas en canal con ventana deslizante.
canal: 2090...17 espigas detectadas en canal con ventana deslizante.
canal: 2091...18 espigas detectadas en canal con ventana deslizante.
canal: 2092...11 espigas detectadas en canal con ventana deslizante.
canal: 2027...12 espigas detectadas en canal con ventana deslizante.
canal: 2028...11 espigas detectadas en canal con ventana deslizante.
canal: 1963...14 espigas detectadas en canal con ventana deslizante.
canal: 1898...22 espigas detectadas en canal con ventana deslizante.
canal: 1899...16 espigas detectadas en canal con ventana deslizante.
canal: 1962...11 espigas detectadas en canal con ventana deslizante.
canal: 2026...13 espigas detectadas en canal con ventana deslizante.
canal: 2025...10 espigas detectadas en canal con ventana deslizante.
canal: 2089...16 espigas detectadas 

canal: 999...15 espigas detectadas en canal con ventana deslizante.
canal: 936...15 espigas detectadas en canal con ventana deslizante.
canal: 872...8 espigas detectadas en canal con ventana deslizante.
canal: 873...16 espigas detectadas en canal con ventana deslizante.
canal: 1001...14 espigas detectadas en canal con ventana deslizante.
canal: 938...16 espigas detectadas en canal con ventana deslizante.
canal: 1002...22 espigas detectadas en canal con ventana deslizante.
canal: 1066...10 espigas detectadas en canal con ventana deslizante.
canal: 1003...18 espigas detectadas en canal con ventana deslizante.
canal: 1131...15 espigas detectadas en canal con ventana deslizante.
canal: 1068...11 espigas detectadas en canal con ventana deslizante.
canal: 1132...15 espigas detectadas en canal con ventana deslizante.
canal: 1004...11 espigas detectadas en canal con ventana deslizante.
canal: 1005...8 espigas detectadas en canal con ventana deslizante.
canal: 940...17 espigas detectadas en can

canal: 298...22 espigas detectadas en canal con ventana deslizante.
canal: 233...15 espigas detectadas en canal con ventana deslizante.
canal: 297...8 espigas detectadas en canal con ventana deslizante.
canal: 361...9 espigas detectadas en canal con ventana deslizante.
canal: 362...10 espigas detectadas en canal con ventana deslizante.
canal: 425...14 espigas detectadas en canal con ventana deslizante.
canal: 426...12 espigas detectadas en canal con ventana deslizante.
canal: 489...15 espigas detectadas en canal con ventana deslizante.
canal: 424...11 espigas detectadas en canal con ventana deslizante.
canal: 488...21 espigas detectadas en canal con ventana deslizante.
canal: 552...15 espigas detectadas en canal con ventana deslizante.
canal: 487...19 espigas detectadas en canal con ventana deslizante.
canal: 551...14 espigas detectadas en canal con ventana deslizante.
canal: 615...13 espigas detectadas en canal con ventana deslizante.
canal: 550...13 espigas detectadas en canal con ve

...

canal: 2300...10 espigas detectadas en canal con ventana deslizante.
canal: 2235...19 espigas detectadas en canal con ventana deslizante.
canal: 2236...9 espigas detectadas en canal con ventana deslizante.
canal: 2171...11 espigas detectadas en canal con ventana deslizante.
canal: 2172...14 espigas detectadas en canal con ventana deslizante.
canal: 2107...22 espigas detectadas en canal con ventana deslizante.
canal: 2108...19 espigas detectadas en canal con ventana deslizante.
canal: 2043...12 espigas detectadas en canal con ventana deslizante.
canal: 2044...16 espigas detectadas en canal con ventana deslizante.
canal: 1979...17 espigas detectadas en canal con ventana deslizante.
canal: 1980...5 espigas detectadas en canal con ventana deslizante.
canal: 1915...16 espigas detectadas en canal con ventana deslizante.
canal: 1916...19 espigas detectadas en canal con ventana deslizante.
canal: 1851...13 espigas detectadas en canal con ventana deslizante.
canal: 1852...9 espigas detectadas e

canal: 2617...11 espigas detectadas en canal con ventana deslizante.
canal: 2552...8 espigas detectadas en canal con ventana deslizante.
canal: 2616...14 espigas detectadas en canal con ventana deslizante.
canal: 2680...10 espigas detectadas en canal con ventana deslizante.
canal: 2681...9 espigas detectadas en canal con ventana deslizante.
canal: 2744...16 espigas detectadas en canal con ventana deslizante.
canal: 2679...15 espigas detectadas en canal con ventana deslizante.
canal: 2743...12 espigas detectadas en canal con ventana deslizante.
canal: 2807...6 espigas detectadas en canal con ventana deslizante.
canal: 2742...20 espigas detectadas en canal con ventana deslizante.
canal: 2806...9 espigas detectadas en canal con ventana deslizante.
canal: 2870...8 espigas detectadas en canal con ventana deslizante.
canal: 2805...11 espigas detectadas en canal con ventana deslizante.
canal: 2869...17 espigas detectadas en canal con ventana deslizante.
canal: 2933...16 espigas detectadas en 

canal: 1077...17 espigas detectadas en canal con ventana deslizante.
canal: 1078...13 espigas detectadas en canal con ventana deslizante.
canal: 1141...11 espigas detectadas en canal con ventana deslizante.
canal: 1142...14 espigas detectadas en canal con ventana deslizante.
canal: 1205...13 espigas detectadas en canal con ventana deslizante.
canal: 1206...15 espigas detectadas en canal con ventana deslizante.
canal: 1269...11 espigas detectadas en canal con ventana deslizante.
canal: 1270...9 espigas detectadas en canal con ventana deslizante.
canal: 1333...11 espigas detectadas en canal con ventana deslizante.
canal: 1334...10 espigas detectadas en canal con ventana deslizante.
canal: 1397...12 espigas detectadas en canal con ventana deslizante.
canal: 1398...8 espigas detectadas en canal con ventana deslizante.
canal: 1461...12 espigas detectadas en canal con ventana deslizante.
canal: 1462...16 espigas detectadas en canal con ventana deslizante.
canal: 1525...13 espigas detectadas 

canal: 2986...16 espigas detectadas en canal con ventana deslizante.
canal: 2923...12 espigas detectadas en canal con ventana deslizante.
canal: 2858...13 espigas detectadas en canal con ventana deslizante.
canal: 2859...13 espigas detectadas en canal con ventana deslizante.
canal: 2860...11 espigas detectadas en canal con ventana deslizante.
canal: 2795...11 espigas detectadas en canal con ventana deslizante.
canal: 2796...8 espigas detectadas en canal con ventana deslizante.
canal: 2797...5 espigas detectadas en canal con ventana deslizante.
canal: 2732...9 espigas detectadas en canal con ventana deslizante.
canal: 2733...12 espigas detectadas en canal con ventana deslizante.
canal: 2734...6 espigas detectadas en canal con ventana deslizante.
canal: 2798...10 espigas detectadas en canal con ventana deslizante.
canal: 2735...13 espigas detectadas en canal con ventana deslizante.
canal: 2670...7 espigas detectadas en canal con ventana deslizante.
canal: 2671...13 espigas detectadas en 

canal: 2667...12 espigas detectadas en canal con ventana deslizante.
canal: 2731...11 espigas detectadas en canal con ventana deslizante.
canal: 2666...9 espigas detectadas en canal con ventana deslizante.
canal: 2730...13 espigas detectadas en canal con ventana deslizante.
canal: 2794...15 espigas detectadas en canal con ventana deslizante.
canal: 2729...14 espigas detectadas en canal con ventana deslizante.
canal: 2793...7 espigas detectadas en canal con ventana deslizante.
canal: 2857...13 espigas detectadas en canal con ventana deslizante.
canal: 2792...10 espigas detectadas en canal con ventana deslizante.
canal: 2856...12 espigas detectadas en canal con ventana deslizante.
canal: 2791...12 espigas detectadas en canal con ventana deslizante.
canal: 2855...14 espigas detectadas en canal con ventana deslizante.
canal: 2919...17 espigas detectadas en canal con ventana deslizante.
canal: 2854...10 espigas detectadas en canal con ventana deslizante.
canal: 2918...12 espigas detectadas 

In [8]:
save( "spikes_140116s2cut2_evento_2.jld", "spikes", AllSpikes );

In [ ]:
#=
sr = 7022; # sampling rate
stdmin = 3; # umbral de deteccion 
fcut = 350; # frecuancias de corte, minima
frames = 1; # ms de separacion entre posibles espigas
i = 1#:length(channels4)
channel = rawdata[ channels4[ i ], : ];
x1, index1 = get_spikes_single( channel, sr, stdmin, fcut, frames );
println(
    "canal: ", channels4[ i ],"...",length(index1)," espigas detectadas en canal completo." );
=#